<a href="https://colab.research.google.com/github/chauhanac/DSprograms/blob/master/Text_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Get the data: The Wonderful Wizard of Oz, by L. Frank Baum
!wget https://www.gutenberg.org/files/55/55.txt -O book.txt

--2021-01-22 08:18:21--  https://www.gutenberg.org/files/55/55.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 237535 (232K) [text/plain]
Saving to: ‘book.txt’

book.txt            100%[===================>] 231.97K   994KB/s    in 0.2s    

2021-01-22 08:18:21 (994 KB/s) - ‘book.txt’ saved [237535/237535]



In [2]:
text = open('book.txt', 'r', encoding= 'utf-8').read()
text = text.lower()

In [3]:
text

'\ufeffthe project gutenberg ebook of the wonderful wizard of oz, by l. frank baum\n\nthis ebook is for the use of anyone anywhere in the united states and most\nother parts of the world at no cost and with almost no restrictions\nwhatsoever.  you may copy it, give it away or re-use it under the terms of\nthe project gutenberg license included with this ebook or online at\nwww.gutenberg.org.  if you are not located in the united states, you\'ll have\nto check the laws of the country where you are located before using this ebook.\n\ntitle: the wonderful wizard of oz\n\nauthor: l. frank baum\n\nrelease date: february, 1993 [ebook #55]\n[most recently updated: october 19, 2020]\n\nlanguage: english\n\ncharacter set encoding: utf-8\n\n*** start of this project gutenberg ebook the wonderful wizard of oz ***\n\n[illustration]\n\n\n\n\nthe wonderful wizard of oz\n\nby l. frank baum\n\n\nthis book is dedicated to my good friend & comrade\nmy wife\nl.f.b.\n\n\ncontents\n\n introduction\n chapte

In [4]:
sentences = text.split('\n')
print(sentences)

['\ufeffthe project gutenberg ebook of the wonderful wizard of oz, by l. frank baum', '', 'this ebook is for the use of anyone anywhere in the united states and most', 'other parts of the world at no cost and with almost no restrictions', 'whatsoever.  you may copy it, give it away or re-use it under the terms of', 'the project gutenberg license included with this ebook or online at', "www.gutenberg.org.  if you are not located in the united states, you'll have", 'to check the laws of the country where you are located before using this ebook.', '', 'title: the wonderful wizard of oz', '', 'author: l. frank baum', '', 'release date: february, 1993 [ebook #55]', '[most recently updated: october 19, 2020]', '', 'language: english', '', 'character set encoding: utf-8', '', '*** start of this project gutenberg ebook the wonderful wizard of oz ***', '', '[illustration]', '', '', '', '', 'the wonderful wizard of oz', '', 'by l. frank baum', '', '', 'this book is dedicated to my good friend & 

In [5]:
import tensorflow as tf
import keras
import string
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences


In [6]:
punc = string.punctuation
tokenizer = Tokenizer(filters=punc,oov_token='<UNK>')

In [7]:
tokenizer.fit_on_texts(sentences)

In [8]:
vocab_size = len(tokenizer.word_index) + 1
vocab_size

3629

In [9]:
sequences = tokenizer.texts_to_sequences(sentences)
print(sequences[0])

[2071, 80, 71, 451, 6, 2, 223, 154, 6, 44, 60, 790, 877, 878]


In [10]:
input_seq = []

for seq in sequences:
  for i in range(1,len(seq)):
    n_gram_sequences = seq[:i+1]
    input_seq.append(n_gram_sequences)


print(input_seq[0], input_seq[1], input_seq[2])

[2071, 80] [2071, 80, 71] [2071, 80, 71, 451]


In [11]:
max_sequence_length = max([len(seq) for seq in input_seq])
max_sequence_length

18

In [12]:
padded_sequences = pad_sequences(input_seq,maxlen=max_sequence_length)
print(padded_sequences[0],padded_sequences[1],padded_sequences[2])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0 2071   80] [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 2071   80   71] [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
 2071   80   71  451]


In [13]:
import numpy as np
padded_sequences = np.array(padded_sequences)

In [14]:
x = padded_sequences[:,:-1]
labels = padded_sequences[:,-1]

In [15]:
y = tf.keras.utils.to_categorical(labels, num_classes=vocab_size)
y.shape

(40273, 3629)

In [16]:
x.shape

(40273, 17)

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional,LSTM,Dense
from tensorflow.keras.optimizers import Adam

In [18]:
model = Sequential()
model.add(Embedding(vocab_size,100,input_length=max_sequence_length-1))
model.add(Bidirectional(LSTM(256)))
model.add(Dense(vocab_size,activation='softmax'))
adam = Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy',optimizer = adam, metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 17, 100)           362900    
_________________________________________________________________
bidirectional (Bidirectional (None, 512)               731136    
_________________________________________________________________
dense (Dense)                (None, 3629)              1861677   
Total params: 2,955,713
Trainable params: 2,955,713
Non-trainable params: 0
_________________________________________________________________


In [19]:
#initializing callback for EarlyStopping; Keeping threshold as 1%
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor = 'acc', min_delta=0.01)

In [20]:
model.fit(x,y,epochs=100,verbose=1,batch_size=512,callbacks=[es])

Epoch 1/100
79/79 [==============================] - 6s 27ms/step - loss: 6.4929 - acc: 0.0658
Epoch 2/100
79/79 [==============================] - 2s 27ms/step - loss: 5.0851 - acc: 0.1499
Epoch 3/100
79/79 [==============================] - 2s 27ms/step - loss: 4.3088 - acc: 0.2049
Epoch 4/100
79/79 [==============================] - 2s 27ms/step - loss: 3.7196 - acc: 0.2443
Epoch 5/100
79/79 [==============================] - 2s 26ms/step - loss: 3.1410 - acc: 0.3144
Epoch 6/100
79/79 [==============================] - 2s 27ms/step - loss: 2.6362 - acc: 0.3940
Epoch 7/100
79/79 [==============================] - 2s 27ms/step - loss: 2.2096 - acc: 0.4773
Epoch 8/100
79/79 [==============================] - 2s 27ms/step - loss: 1.8380 - acc: 0.5504
Epoch 9/100
79/79 [==============================] - 2s 27ms/step - loss: 1.5605 - acc: 0.6122
Epoch 10/100
79/79 [==============================] - 2s 27ms/step - loss: 1.3392 - acc: 0.6629
Epoch 11/100
79/79 [=============================

In [21]:
seed_text = "With these words the Witch fell down in a brown, melted, shapeless mass \
and began to spread over the clean boards of the kitchen floor. Seeing \
that she had really melted away to nothing, Dorothy drew another bucket \
of water and threw it over the mess. She then swept it all out the\
door."
next_words = 100

for _ in range(next_words):
  sequence = tokenizer.texts_to_sequences([seed_text])
  padded = pad_sequences(sequence, maxlen = max_sequence_length-1)
  predicted = np.argmax(model.predict(padded), axis=-1)
  output_word = ''
  for word, index in tokenizer.word_index.items():

    if index==predicted:
      output_word = word
      break
    
  seed_text+= ' '+output_word

print(seed_text)

With these words the Witch fell down in a brown, melted, shapeless mass and began to spread over the clean boards of the kitchen floor. Seeing that she had really melted away to nothing, Dorothy drew another bucket of water and threw it over the mess. She then swept it all out thedoor. to the emerald city her and she had not been by the people so that the wicked witch had asked her a heart i was in love i could i have had a very good and comrade again and so they said ran to the scarecrow but he was so he said “it was a good witch of the great as i have given it to the palace of the tin woodman and the scarecrow began to grumble a bit as not to think about me to find a chair with his little so that will as not to have


In [22]:
import matplotlib.pyplot as plt
history = model.history
acc = history.history['acc']
loss = history.history['loss']

epochs = range(len(acc))
plt.plot(epochs,acc,'b',label = 'Training Accuracy')
plt.title('Training Accuracy')
plt.figure()

plt.plot(epochs,acc,'r',label = 'Training Loss')
plt.title('Training Loss')
plt.legend()

plt.show()

KeyError: ignored